In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from transformers import TFBertModel, BertConfig, BertTokenizer, BertModel, DistilBertModel
from tokenization_kobert import KoBertTokenizer

In [2]:
# 데이터셋 로드
# Step 1: 데이터셋 불러오기
train_data = pd.read_csv("9keywords_train.csv")
# 컬럼 이름 변경
train_data.drop(['감정_대분류'], axis=1, inplace=True)
train_data = train_data.rename(columns={'상황키워드': 'label', '사람문장': 'document'})
train_data.loc[(train_data['label']=='대인관계'), 'label'] = 0
train_data.loc[(train_data['label']=='진로,취업,직장'), 'label'] = 1
train_data.loc[(train_data['label']=='연애,결혼,출산'), 'label'] = 2 
train_data.loc[(train_data['label']=='가족관계'), 'label'] = 3 
train_data.loc[(train_data['label']=='학업 및 진로'), 'label'] = 4 
train_data.loc[(train_data['label']=='학교폭력/따돌림'), 'label'] = 5
train_data.loc[(train_data['label']=='재정,은퇴,노후준비'), 'label'] = 6
train_data.loc[(train_data['label']=='직장, 업무 스트레스'), 'label'] = 7
train_data.loc[(train_data['label']=='건강,죽음'), 'label'] = 8

train_data = train_data[["document", "label"]]
train_data = train_data.reset_index(drop=True) # 인덱스 리셋

# 라벨링
label_dict = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8}
train_data['label'] = train_data['label'].apply(lambda x: label_dict[str(x)])

# tokenizer 준비
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# 입력 데이터 전처리 함수
def preprocess_sentence(sentence, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(
        text=sentence,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_attention_mask=True,
        truncation=True
    )
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    return input_id, attention_mask

# 입력 데이터 전처리
MAX_LEN = 50
input_ids = []
attention_masks = []
for sentence in train_data['document']:
    input_id, attention_mask = preprocess_sentence(sentence, MAX_LEN)
    input_ids.append(input_id)
    attention_masks.append(attention_mask)

# 입력 데이터를 numpy array로 변환
input_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)
labels = np.array(train_data['label'])

# 모델 생성 및 훈련
bert_model = TFBertModel.from_pretrained('monologg/kobert', from_pt=True)
input_layer = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
attention_layer = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')
output_layer = bert_model([input_layer, attention_layer])[1]
output_layer = tf.keras.layers.Dense(9, activation='softmax')(output_layer)
model = tf.keras.models.Model(inputs=[input_layer, attention_layer], outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit([input_ids, attention_masks], labels, epochs=30, batch_size=32, validation_split=0.2)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
1291/1291 [==============================] - 277s 206ms/step - loss: 0.9443 - accuracy: 0.6643 - val_loss: 1.4277 - val_accuracy: 0.5279
Epoch 2/30
1291/1291 [==============================] - 217s 168ms/step - loss: 0.6913 - accuracy: 0.7565 - val_loss: 1.4310 - val_accuracy: 0.5606
Epoch 3/30
1291/1291 [==============================] - 254s 197ms/step - loss: 0.5852 - accuracy: 0.7935 - val_loss: 1.4311 - val_accuracy: 0.5445
Epoch 4/30
1291/1291 [==============================] - 281s 218ms/step - loss: 0.4844 - accuracy: 0.8305 - val_loss: 1.6464 - val_accuracy: 0.5437
Epoch 5/30
1291/1291 [==============================] - 206s 160ms/step - loss: 0.3870 - accuracy: 0.8674 - val_loss: 1.9006 - val_accuracy: 0.5096
Epoch 6/30
1291/1291 [==============================] - 206s 159ms/step - loss: 0.3040 - accuracy: 0.8994 - val_loss: 1.9803 - val_accuracy: 0.5227
Epoch 7/30
1291/1291 [==============================] - 206s 160ms/step - loss: 0.2352 - accuracy: 0.9221 - val_

In [3]:
# 모델 저장
model.save('./model_save/kobert_situation_epochs.h5')

In [4]:
# Tokenizer 저장
import pickle

with open('tokenizer_situation_epochs.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
from tensorflow.keras.models import load_model

In [5]:
# Tokenizer 불러오기
import pickle

with open('tokenizer_situation_epochs.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [8]:
# 모델 불러오기
model = load_model('./model_save/kobert_situation_epochs.h5', custom_objects={'TFBertModel': TFBertModel})


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Okt
import re, pickle
import numpy as np

class model:
  def __init__(self):
    self.max_len = 50
    self.stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    self.first_model = load_model('MH-1.0.1.h5')
    self.second_model = load_model('MH-2.0.1.h5')
    with open('tokenizer.pickle', 'rb') as handle:
      self.tokenizer = pickle.load(handle)
    self.okt = Okt()

    

  def sentiment_predict(self, new_sentence):

    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = self.okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in self.stopwords] # 불용어 제거
    encoded = self.tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = self.max_len) # 패딩
    score = float(np.argmax(self.first_model.predict(pad_new), axis=-1)) # 예측
    return score

  def circumstance_predict(self, new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = self.okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in self.stopwords] # 불용어 제거
    encoded = self.tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = self.max_len) # 패딩
    score = float(np.argmax(self.second_model.predict(pad_new), axis=-1)) # 예측
    return score

In [9]:
# Step 1: 데이터셋 불러오기
test_data = pd.read_csv("9keywords_test.csv")
# 컬럼 이름 변경
test_data.drop(['감정_대분류'], axis=1, inplace=True)
test_data = test_data.rename(columns={'상황키워드': 'label', '사람문장': 'document'})
test_data.loc[(test_data['label']=='대인관계'), 'label'] = 0
test_data.loc[(test_data['label']=='진로,취업,직장'), 'label'] = 1
test_data.loc[(test_data['label']=='연애,결혼,출산'), 'label'] = 2 
test_data.loc[(test_data['label']=='가족관계'), 'label'] = 3 
test_data.loc[(test_data['label']=='학업 및 진로'), 'label'] = 4 
test_data.loc[(test_data['label']=='학교폭력/따돌림'), 'label'] = 5
test_data.loc[(test_data['label']=='재정,은퇴,노후준비'), 'label'] = 6
test_data.loc[(test_data['label']=='직장, 업무 스트레스'), 'label'] = 7
test_data.loc[(test_data['label']=='건강,죽음'), 'label'] = 8
test_data = test_data[["document", "label"]]
test_data = test_data.reset_index(drop=True) # 인덱스 리셋

# 라벨링
label_dict = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8}
test_data['label'] = test_data['label'].apply(lambda x: label_dict[str(x)])

# tokenizer 준비
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# 입력 데이터 전처리 함수
def preprocess_sentence(sentence, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(
        text=sentence,
        add_special_tokens=True,
        max_length=MAX_LEN,
        pad_to_max_length=True,
        return_attention_mask=True
    )
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    return input_id, attention_mask

# 입력 데이터 전처리
MAX_LEN = 50
input_ids = []
attention_masks = []
for sentence in test_data['document']:
    input_id, attention_mask = preprocess_sentence(sentence, MAX_LEN)
    input_ids.append(input_id)
    attention_masks.append(attention_mask)

# 입력 데이터를 numpy array로 변환
input_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)
labels = np.array(test_data['label'])

# 모델 예측
predictions = model.predict([input_ids, attention_masks])
predicted_labels = np.argmax(predictions, axis=1)

# 정확도 계산
accuracy = np.sum(predicted_labels == labels) / len(labels)
print("Accuracy:", accuracy)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\ykh98\anaconda3\envs\tf2\lib\site-packages\transformers\tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max leng

208/208 [==============================] - 13s 55ms/step
Accuracy: 0.8272850474326156
